# Tools and Routing

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain_core.tools import tool

In [4]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [5]:
search.name

'search'

In [6]:
search.description

'Search for weather online'

In [7]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [8]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [10]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [11]:
search.run("sf")

'42f'

In [41]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [42]:
get_current_temperature.name

'get_current_temperature'

In [43]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [44]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

In [45]:
# !pip install -U langchain-core

In [46]:
from langchain_core.utils.function_calling import convert_to_openai_function

In [47]:
convert_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [49]:
get_current_temperature({"latitude": 21.8314, "longitude": 76.3498})

C:\Users\patil\AppData\Local\Temp\ipykernel_19840\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 37.2°C'

In [51]:
# !pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=1f1495b0a311d0a688a3828c514f9b9c4d2c7e327ebac242284c43dec0612d45
  Stored in directory: c:\users\patil\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [52]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [53]:
search_wikipedia.name

'search_wikipedia'

In [54]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [55]:
convert_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [56]:
search_wikipedia({"query": "langchain"})

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information. With RAG, LLMs do not respond to user queries until they refer to a specified set of documents. These documents supplement information from the LLM\'s pre-existing training data. This allows LLMs to use domain-specific and/or updated information that is not available in the training data. For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources.\nRAG improves large language models (LLMs) by

In [69]:
# from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# from langchain.utilities.openapi import OpenAPISpec

from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain_community.utilities.openapi import OpenAPISpec

In [70]:
text = """
{
  "openapi": "3.1.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [71]:
spec = OpenAPISpec.from_text(text)

AttributeError: 'super' object has no attribute 'parse_obj'

In [68]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

NameError: name 'spec' is not defined

In [ ]:
pet_openai_functions

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [ ]:
model.invoke("what are three pets names")

In [ ]:
model.invoke("tell me about pet with id 42")

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [73]:
from langchain_openai import ChatOpenAI

functions = [
    convert_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [74]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 105, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhF7Ozdg5A7TIBChpLAZ9LGchrIIj', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--33a790d8-58ed-400a-a8a5-5c2b1255eab2-0', usage_metadata={'input_tokens': 105, 'output_tokens': 25, 'total_tokens': 130, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [75]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 101, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhF7Q303y70wH3fyTv604ewi2jXGR', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--d4d9c543-fb63-418a-beab-3a2a619a1236-0', usage_metadata={'input_tokens': 101, 'output_tokens': 16, 'total_tokens': 117, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [76]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [77]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 113, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhF7TRgeh6lIaOjskmwpxawApMpPa', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--4beeb6cb-be87-4017-897b-df9c21befcf9-0', usage_metadata={'input_tokens': 113, 'output_tokens': 25, 'total_tokens': 138, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [78]:
from langchain.agents.output_parsers.openai_functions import OpenAIFunctionsAgentOutputParser


In [79]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [80]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [81]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [82]:
result.tool

'get_current_temperature'

In [83]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [84]:
get_current_temperature(result.tool_input)

C:\Users\patil\AppData\Local\Temp\ipykernel_19840\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 11.3°C'

In [85]:
result = chain.invoke({"input": "hi!"})

In [86]:
type(result)

langchain_core.agents.AgentFinish

In [87]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [88]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [89]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [90]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

C:\Users\patil\AppData\Local\Temp\ipykernel_19840\4059959254.py:32: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [91]:
result

'The current temperature is 11.3°C'

In [92]:
result = chain.invoke({"input": "What is langchain?"})

In [93]:
result

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information. With RAG, LLMs do not respond to user queries until they refer to a specified set of documents. These documents supplement information from the LLM\'s pre-existing training data. This allows LLMs to use domain-specific and/or updated information that is not available in the training data. For example, this helps LLM-based chatbots access internal company data or generate responses based on authoritative sources.\nRAG improves large language models (LLMs) by

In [94]:
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'